In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
#os.chdir("../Datastets")
#myData = pd.read_csv(r'C:\Users\Paul\Projects\Skill_Notebooks\Module_5\DataSets\mycar.csv')
myData = pd.read_csv(r'C:\Users\wangshu202040\Skill_Notes\Module_5\DataSets\mycar.csv')

In [ ]:
myData.head()

In [ ]:
myData.shape

In [ ]:
X = myData.iloc[:,:-1].values
Y = myData.iloc[:,1].values

In [ ]:
X_train,X_test,Y_train, Y_test = train_test_split(X,Y,test_size = 0.3)

In [ ]:
from sklearn.linear_model import LinearRegression
myModel = LinearRegression()
myModel.fit(X_train,Y_train)

In [ ]:
y_pred = myModel.predict(X_test)
y_pred

# <span style="color: orange; font-weight: bold; font-size:16pt">3A.5 Линейная регрессия. Предобработка</span>

In [ ]:
colors = ['#50248f', '#38d1ff']
sns.palplot(sns.color_palette(colors))

In [ ]:
data = pd.read_csv(r'C:/Users/wangshu202040/Skill_Notes/Module_5/DataSets/data_flats2.csv', sep = ';')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
cols = data.columns
fig, ax = plt.subplots(figsize=(7, 7))
sns.heatmap(data[cols].isnull(), cmap=sns.color_palette(colors))

У нас пропуски по сути есть только в одном признаке — жилой площади. Просто не будем брать её в модель.

Далее всегда есть смысл посмотреть на распределение нашей целевой переменной и понять, какие значения мы будем предсказывать:

In [ ]:
data.price_doc.hist();

У нашего распределения есть проблема — слишком сильный перепад. Много квартир в среднем сегменте, но очень мало дорогих квартир. На практике часто в таких случаях логорифмируют переменную, чтобы уменьшить перепады и сгладить хвост.

In [ ]:
data['price_doc'] = data['price_doc'].apply(lambda w: np.log(w+1))
data.price_doc.hist();

Теперь займемся отбором признаков.  Для начала нам надо проверить, нет ли мультиколлинеарности — сильной взаимосвязи между независимыми признаками. Для этого построим матрицу корреляций для признаков:

In [ ]:
sns.set(font_scale=1)
plt.subplots(figsize = (12,12))
sns.heatmap(data.corr(),square = True,
           annot = True, fmt = '.1f',linewidths=.1,cmap='RdBu')

In [ ]:
cols_to_del = ['id','preschool_education_centers_raion','kindergarten_km','park_km','kremlin_km','life_sq']

In [ ]:
data1 = data.drop(cols_to_del,axis=1)

In [ ]:
data1 = data1.dropna()

In [ ]:
# X = data1.iloc[:,:-1].values
# Y = data1.iloc[:,1].values

X = data1.drop(['price_doc'],axis = 1)
Y = data1['price_doc']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,Y_train, Y_test = train_test_split(X,Y,test_size = 0.2,random_state=77)

## Feature Scaling

In [ ]:
from sklearn.preprocessing import RobustScaler

In [ ]:
rc = RobustScaler()
X_train_tr = rc.fit_transform(X_train)
X_test_tr = rc.transform(X_test)

## Model

In [ ]:
from sklearn.linear_model import LinearRegression
myModel = LinearRegression()
myModel.fit(X_train_tr,Y_train)

In [ ]:
from sklearn.metrics import mean_squared_error as mse

In [ ]:
Y_pred = myModel.predict(X_test_tr)
MSE = np.round(mse(np.exp(Y_test)-1,np.exp(Y_pred)-1),0)

In [ ]:
print(f'MSE = {MSE}')

# <span style="color: orange; font-weight: bold; font-size:16pt">3A.6. Линейная регрессия. Практика №1</span>

## 1. Линейная регрессия. Реализация

In [19]:
data = load_boston()
data['data'].shape

(506, 13)

### 1.1. Реализация линейной регрессии с использованием матричных операций

Линейная регрессия выражается следующей зависимостью:
$$y=X\theta+\epsilon,$$
где $X$ — матрица объекты-признаки, $y$ — вектор целевых значений, соответствующих $X$, $\theta$ — параметр линейной регрессии, $\epsilon$ — некоторый шум.

Из данного следует выражение для $\theta$ как:
$$X^Ty=X^TX\theta \rightarrow \theta=(X^TX)^{-1}X^Ty$$

Реализуем выражение для $\theta$ с помощью операций линейной алгебры библиотеки Numpy:

In [ ]:
def linreg_linear(X,y):
    theta = np.linalg.solve(X.T@X, X.T@y)
    return theta

In [ ]:
# Подготовить данные

X, y = data['data'], data['target']

X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])

In [ ]:
# Вычислить параметр theta
theta = linreg_linear(X,y)

In [ ]:
theta.shape

In [ ]:
# Сделать предсказания для тренировочной выборки
y_pred = np.dot(X,theta)

In [ ]:
def print_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true,y_pred)
    rmse = np.sqrt(mse)
    print (f'MSE = {mse:.2f}, RMSE= {rmse:.2f}')

In [ ]:
# Посчитать значение ошибок MSE и RMSE для тренировочных данных
print_regression_metrics(y, y_pred)

In [ ]:
plt.hist(y);

In [ ]:
# plot data and model prediction
N=len(y)
plt.plot(np.arange(1,N+1),y,'bs-',label='Data')
plt.plot(np.arange(1,N+1),y_pred,'ro--',label='Model pred.')

plt.legend()
plt.show()

In [ ]:
# Разбить выборку на train/valid, вычислить theta,
# сделать предсказания и посчитать ошибки MSE и RMSE

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)
theta = linreg_linear(X_train, y_train)
y_pred = X_valid.dot(theta)
y_train_pred = X_train.dot(theta)

In [ ]:
print_regression_metrics(y_valid, y_pred)
print_regression_metrics(y_train, y_train_pred)

#### Задание 3.6.2

Постройте модель при помощи sklearn. Используйте параметры по умолчанию, обучите на всей выборке и посчитайте RMSE.
Ответ округлите до сотых. Пример ввода: 5.55.

In [18]:
X, y = data['data'], data['target']

X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])

NameError: name 'data' is not defined

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X,y)
y_pred = lr.predict(X)
print_regression_metrics(y,y_pred)

In [ ]:
df = pd.DataFrame(X)
A = df.describe().reset_index()
A.loc[A['index'] == 'std'].max()

 #### Задание 3.6.4

Обучите регрессию без дополнительного столбца единиц. Какой получился RMSE?
Ответ округлите до сотых. Пример ввода: 5.55.

In [16]:
from sklearn.linear_model import LinearRegression

In [15]:
# Подготовить данные
X, y = data['data'], data['target']


NameError: name 'data' is not defined

In [ ]:
lr = LinearRegression()
lr.fit(X,y)
y_pred = lr.predict(X)
print_regression_metrics(y,y_pred)

#### Задание 3.6.5

Очистите данные от строк, где значение признака  меньше . Какой получился RMSE?
Ответ округлите до сотых. Пример ввода: 5.55.

In [ ]:
X = pd.DataFrame(data = data['data'],columns=data['feature_names'])
y = data['target']
X['ones'] = 1

In [ ]:
y = y[X['B']>50]
X = X[X['B']>50]

In [ ]:
lr = LinearRegression()
lr.fit(X,y)
y_pred = lr.predict(X)
print_regression_metrics(y,y_pred)

#### Задание 3.6.6

Нормализуйте признаки и обучите линейную регрессию матричным методом. Какой получился RMSE?
Ответ округлите до сотых. Пример ввода: 5.55.

In [ ]:
X = pd.DataFrame(data = data['data'],columns=data['feature_names'])
y = data['target']
X['ones'] = 1

In [ ]:
rc = RobustScaler()
X_train_tr = rc.fit_transform(X)

In [ ]:
# Вычислить параметр theta
theta = linreg_linear(X,y)

In [ ]:
y_pred = X@theta

In [ ]:
print_regression_metrics(y,y_pred)

### 1.2. Реализация линейной регрессии с использованием методов оптимизации

Для реализации линейной регрессии с помощью методов оптимизации будем использовать функцию ошибки **среднего квадратичного** ([Mean Squared Error](https://en.wikipedia.org/wiki/Mean_squared_error)), которая является выпуклой функцией в n-мерном пространстве $\mathbb{R}^n$ и в общем виде выглядит следующим образом:
$$MSE = \frac{1}{n} * \sum_{i=1}^{n}{(y_i - a(x_i))^2}.$$
Здесь $x_i$ — вектор-признак $i$-го объекта обучающей выборки, $y_i$ — истинное значение для $i$-го объекта, $a(x)$ — алгоритм, предсказывающий для данного объекта $x$ целевое значение (иначе говоря, предсказанное значение), $n$ — кол-во объектов в выборке.

В случае линейной регрессии $MSE$ представляется как:
$$MSE(X, y, \theta) = \frac{1}{2n} * \sum_{i=1}^{n}{(y_i - \theta^Tx_i)^2} = \frac{1}{2n} \lVert{y - X\theta}\rVert_{2}^{2}=\frac{1}{2n} (y - X\theta)^T(y - X\theta),$$
где $\theta$ — параметр модели линейной регрессии, $X$ — матрица объекты-признаки, $y$ - вектор истинных значений, соответствующих $X$.

Возьмем первый вариант представления функции ошибки и посчитаем ее градиент по параметру $\theta$, предварительно переименовав $MSE$ в $L$:
$$L=\frac{1}{2n} * \sum_{i=1}^{n}{(y_i - \theta^Tx_i)^2}$$
$$\nabla L = \frac{1}{n}\sum_{i=1}^{n}{(\theta^Tx_i - y_i) \cdot x_i} = \frac{1}{n}X^T(X\theta - y)$$

In [20]:
# Реализовать функцию вычисления градиента функции MSE

def calc_mse_gradient(X, y, theta):
    n = X.shape[0]
    grad = 1. / n * X.transpose().dot(X.dot(theta) - y)
    
    return grad

In [21]:
# Реализовать функцию, осуществляющую градиентный шаг
# (функция должна содержать параметр величины шага alpha - learning rate)

def gradient_step(theta, theta_grad, alpha):
    return theta - alpha * theta_grad

In [22]:
# Реализовать функцию цикла градиентного спуска с доп. параметрами
# начального вектора theta и числа итераций

def optimize(X, y, grad_func, start_theta, alpha, n_iters):
    theta = start_theta.copy()
    
    for i in range(n_iters):
        theta_grad = grad_func(X, y, theta)
        theta = gradient_step(theta, theta_grad, alpha)
    
    return theta

In [23]:
# Разбить таблицу данных на матрицы X и y
X, y = data['data'], data['target']

# Добавить фиктивный столбец единиц (bias линейной модели)
X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])
m = X.shape[1]

In [24]:
# Оптимизировать параметр линейной регрессии theta на всех данных
theta = optimize(X, y, calc_mse_gradient, np.ones(m), 0.001, 100)

In [25]:
theta

array([7.41647399e+246, 3.32349992e+247, 7.39564172e+247, 8.96295209e+247,
       5.07578059e+245, 4.22030567e+246, 4.63094053e+247, 5.29083888e+248,
       2.65643383e+247, 8.19991211e+247, 3.27135991e+249, 1.38363846e+248,
       2.64323053e+249, 9.88835598e+247])

In [ ]:
X.max(axis=0)

In [ ]:
print(data['feature_names'][np.argmax(X.std(axis=0)) + 1])
print(np.max(X.std(axis=0)))

In [ ]:
# Нормализовать даннные с помощью стандартной нормализации
X, y = data['data'], data['target']
X = (X - X.mean(axis=0)) / X.std(axis=0)

In [ ]:
# Добавить фиктивный столбец единиц (bias линейной модели)
X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])
X.max(axis=0)

In [ ]:
theta = optimize(X, y, calc_mse_gradient, np.ones(m), 0.01, 5000)

In [ ]:
theta

In [ ]:
# Сделать предсказания при полученных параметрах
y_pred = X.dot(theta)

In [ ]:
# Посчитать значение ошибок MSE и RMSE для тренировочных данных
print_regression_metrics(y, y_pred)

In [ ]:
# Разбить выборку на train/valid, оптимизировать theta,
# сделать предсказания и посчитать ошибки MSE и RMSE

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)
theta = optimize(X_train, y_train, calc_mse_gradient, np.ones(m), 0.01, 5000)
y_pred = X_valid.dot(theta)

print_regression_metrics(y_valid, y_pred)

# <span style="color: orange; font-weight: bold; font-size:16pt">3А.7. Линейная регрессия. Практика №2</span>

### 2.1.Когда использовать матричные операции вместо градиентного спуска в линейной регрессии

In [2]:
def print_regression_metrics(y_true, y_pred):
    '''Print the value of errors
    INPUT: target value true,
           Predicted value
    OUTPUT:Value of MSE and RMSE'''
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    print(f'MSE = {mse:.2f}, RMSE = {rmse:.2f}')
    
def prepare_boston_data():
    data = load_boston()
    X, y = data['data'], data['target']
    # Нормализовать даннные с помощью стандартной нормализации
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    # Добавить фиктивный столбец единиц (bias линейной модели)
    X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])
    
    return X, y

Прежде чем начать, обернем написанную нами линейную регрессию методом матричных операций в класс:

In [3]:
class LinRegAlgebra():
    def __init__(self):
        self.theta = None

    def fit(self, X, y):
#         self.theta = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(y)
        self.theta = np.linalg.solve(X.T@X, X.T@y)

    def predict(self, X):
        return X@self.theta

Проведем замеры скорости работы алгоритмов на матричных операциях и на градиентном спуске. Предварительно найдем параметры для метода, основанного на градиентном спуске, так, чтобы значения метрик максимально совпадало со значениями в случае первого алгоритма.

In [4]:
X, y = prepare_boston_data()

In [5]:
linreg_alg = LinRegAlgebra()
linreg_alg.fit(X,y)
y_pred = linreg_alg.predict(X)

# Посчитать значение ошибок MSE и RMSE для тренировочных данных
print_regression_metrics(y,y_pred)

MSE = 21.89, RMSE = 4.68


In [6]:
class RegOptimizer():
    def __init__(self, alpha, n_iters):
        self.theta = None
        self._alpha = alpha
        self._n_iters = n_iters
    
    def gradient_step(self, theta, theta_grad):
        return theta - self._alpha * theta_grad
    
    def grad_func(self, X, y, theta):
        raise NotImplementedError()

    def optimize(self, X, y, start_theta, n_iters):
        theta = start_theta.copy()

        for i in range(n_iters):
            theta_grad = self.grad_func(X, y, theta)
            theta = self.gradient_step(theta, theta_grad)

        return theta
    
    def fit(self, X, y):
        m = X.shape[1]
        start_theta = np.ones(m)
        self.theta = self.optimize(X, y, start_theta, self._n_iters)
        
    def predict(self, X):
        raise NotImplementedError()

In [7]:
class LinReg(RegOptimizer):
    def grad_func(self, X, y, theta):
        n = X.shape[0]
        grad = 1. / n * X.transpose().dot(X.dot(theta) - y)

        return grad
    
    def predict(self, X):
        if self.theta is None:
            raise Exception('You should train the model first')
        
        y_pred = X.dot(self.theta)
        
        return y_pred

In [8]:
linreg_crit = LinReg(0.2,1000)
linreg_crit.fit(X, y)
y_pred = linreg_crit.predict(X)

# Посчитать значение ошибок MSE и RMSE для тренировочных данных
print_regression_metrics(y, y_pred)

MSE = 21.89, RMSE = 4.68


In [9]:
%timeit linreg_alg.fit(X, y)

37.7 µs ± 2.01 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [10]:
%timeit linreg_crit.fit(X, y)

17.2 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Как видно из результатов эксперимента, реализация на матричных операциях опережает реализацию на градиентном спуске в 1000 раз. Но всегда ли это так и какие подводные камни могут быть? Ниже приведен набор случаев, при которых версия с градентным спуском предпочтительнее:

1. Градиентный спуск работает быстрее в случае матриц с большим количеством признаков. Основная по сложности операция — нахождение обратной матрицы $(X^T X)^{-1}$.
1. Нахождение обратной матрицы может также потребовать больше оперативной памяти, что иногда является не приемлемым.
1. Матричные операции могут также проигрывать и в случае небольших объемов данных, но при плохой параллельной реализации или недостаточных ресурсах.
1. Градиентный спуск может быть усовершенствован до так называемого **стохастического градиентного спуска**, при котором данные для оптимизации подгружаются небольшими наборами, что уменьшает требования по памяти.
1. В некоторых случаях (например, в случае линейно-зависимых строк) алгебраический способ решения не будет работать совсем в виду невозможности найти обратную матрицу.

### 2.2. Превращение линейной модели в нелинейную

Нелинейные зависимости в данных встречаются намного чаще линейных. На самом деле простейшая линейная регрессия способна обнаруживать нелинейные зависимости. Для этого необходимо рассмотреть дополнительные признаки, полученные из исходных применением различных нелинейных функций. Возьмем уже знакомый датасет с ценами на квартиры в Бостоне и последовательно станем применять различные функции к исходным признакам:

Boston Data. Attribute Information (in order):
    - CRIM     per capita crime rate by town
    - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
    - INDUS    proportion of non-retail business acres per town
    - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
    - NOX      nitric oxides concentration (parts per 10 million)
    - RM       average number of rooms per dwelling
    - AGE      proportion of owner-occupied units built prior to 1940
    - DIS      weighted distances to five Boston employment centres
    - RAD      index of accessibility to radial highways
    - TAX      full-value property-tax rate per `$10000`
    - PTRATIO  pupil-teacher ratio by town
    - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
    - LSTAT    % lower status of the population
    - MEDV     Median value of owner-occupied homes in $1000's

In [11]:
def prepare_boston_data_new():
    data = load_boston()
    X, y = data['data'], data['target']
    
    X = np.hstack([X, np.sqrt(X[:, 5:6]), X[:, 6:7] ** 3])
    
    # Нормализовать даннные с помощью стандартной нормализации
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    # Добавить фиктивный столбец единиц (bias линейной модели)
    X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])
    
    return X, y

Мы добавили два новых признака:
1. Взяли корень из признака RM (среднее число комнат на сожителя)
1. Возвели в куб значения признака AGE

Это только два примера. Всевозможных комбинаций признаков и примененных к ним функций неограниченное количество.

In [13]:
def train_validate(X, y):
    # Разбить данные на train/valid
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1)

    # Создать и обучить линейную регрессию
    linreg_alg = LinRegAlgebra()
    linreg_alg.fit(X_train, y_train)

    # Сделать предсказания по валидционной выборке
    y_pred = linreg_alg.predict(X_valid)

    # Посчитать значение ошибок MSE и RMSE для валидационных данных
    print_regression_metrics(y_valid, y_pred)

In [14]:
# Подготовить данные без модификации признаков
X, y = prepare_boston_data()
# Провести эксперимент
train_validate(X, y)

MSE = 23.38, RMSE = 4.84


In [15]:
# Подготовить данные с модификации признаков
X, y = prepare_boston_data_new()
# Провести эксперимент
train_validate(X, y)

MSE = 14.28, RMSE = 3.78


#### Задание 3.7.1

Сделайте для градиентного спуска остановку алгоритма, если максимальное из абсолютных значений компонент градиента становится меньше 0.01 . Сравните скорость обучения градиентным спуском и матричными операциями.

На какой итерации останавливается градиентный спуск?

In [18]:
def print_regression_metrics(y_true, y_hat):
    '''Print the error value
    INP: Known Y
    OUT: Predicted Y'''
    mse = mean_squared_error(y_true, y_hat)
    rmse = np.sqrt(mse)
    print(f'MSE = {mse:.2f}, RMSE = {rmse:.2f}')
    
def prepare_boston_data():
    data = load_boston()
    X,y = data['data'], data['target']
    # Нормализовать даннные с помощью стандартной нормализации
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    # Добавить фиктивный столбец единиц (bias линейной модели)
    X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])
    
    return X,y
    

In [22]:
X, y = prepare_boston_data()

In [23]:
class RegOptimizer():
    def __init__(self, alpha, n_iters):
        self.theta = None
        self._alpha = alpha
        self._n_iters = n_iters
    
    def gradient_step(self, theta, theta_grad):
        return theta - self._alpha * theta_grad
    
    def grad_func(self, X, y, theta):
        raise NotImplementedError()
        
    def optimize(self, X, y, start_theta, n_iters):
        theta = start_theta.copy()
        
        for i in range(n_iters):
            theta_grad = self.grad_func(X,y,theta)
            if max(abs(theta_grad)) < 0.01:
                print(max(abs(theta_grad)), f'Num of iter {i}')
                return theta
            theta = self.gradient_step(theta,theta_grad)
            print(f'макс.абс.знач.--{max(abs(theta_grad))}',f'iter--{i}')
        return theta
    
    def fit(self, X, y):
        m = X.shape[1]
        start_theta = np.ones(m)
        self.theta = self.optimize(X, y, start_theta, self._n_iters)
        
    def predict(self, X):
        raise NotImplementedError()

In [24]:
class LinReg(RegOptimizer):
    def grad_func(self, X, y, theta):
        n = X.shape[0]
        grad = 1. / n * X.transpose().dot(X.dot(theta) - y)

        return grad
    
    def predict(self, X):
        if self.theta is None:
            raise Exception('You should train the model first')
        
        y_pred = X.dot(self.theta)
        
        return y_pred

In [25]:
linreg_crit = LinReg(0.2,1000)
linreg_crit.fit(X, y)
y_pred = linreg_crit.predict(X)

# Посчитать значение ошибок MSE и RMSE для тренировочных данных
print_regression_metrics(y, y_pred)

макс.абс.знач.--21.53280632411068 итер--0
макс.абс.знач.--17.226245059288544 итер--1
макс.абс.знач.--13.780996047430838 итер--2
макс.абс.знач.--11.02479683794467 итер--3
макс.абс.знач.--8.819837470355738 итер--4
макс.абс.знач.--7.055869976284591 итер--5
макс.абс.знач.--5.644695981027674 итер--6
макс.абс.знач.--4.515756784822137 итер--7
макс.абс.знач.--3.612605427857711 итер--8
макс.абс.знач.--2.8900843422861677 итер--9
макс.абс.знач.--2.3120674738289324 итер--10
макс.абс.знач.--1.8496539790631477 итер--11
макс.абс.знач.--1.4797231832505182 итер--12
макс.абс.знач.--1.1837785466004147 итер--13
макс.абс.знач.--0.947022837280333 итер--14
макс.абс.знач.--0.757618269824268 итер--15
макс.абс.знач.--0.6060946158594108 итер--16
макс.абс.знач.--0.48487569268753233 итер--17
макс.абс.знач.--0.3879005541500253 итер--18
макс.абс.знач.--0.341222728455631 итер--19
макс.абс.знач.--0.32476160744359206 итер--20
макс.абс.знач.--0.3095605792623588 итер--21
макс.абс.знач.--0.29544878005153236 итер--22
макс.

In [26]:
%timeit linreg_crit.fit(X, y)

макс.абс.знач.--21.53280632411068 итер--0
макс.абс.знач.--17.226245059288544 итер--1
макс.абс.знач.--13.780996047430838 итер--2
макс.абс.знач.--11.02479683794467 итер--3
макс.абс.знач.--8.819837470355738 итер--4
макс.абс.знач.--7.055869976284591 итер--5
макс.абс.знач.--5.644695981027674 итер--6
макс.абс.знач.--4.515756784822137 итер--7
макс.абс.знач.--3.612605427857711 итер--8
макс.абс.знач.--2.8900843422861677 итер--9
макс.абс.знач.--2.3120674738289324 итер--10
макс.абс.знач.--1.8496539790631477 итер--11
макс.абс.знач.--1.4797231832505182 итер--12
макс.абс.знач.--1.1837785466004147 итер--13
макс.абс.знач.--0.947022837280333 итер--14
макс.абс.знач.--0.757618269824268 итер--15
макс.абс.знач.--0.6060946158594108 итер--16
макс.абс.знач.--0.48487569268753233 итер--17
макс.абс.знач.--0.3879005541500253 итер--18
макс.абс.знач.--0.341222728455631 итер--19
макс.абс.знач.--0.32476160744359206 итер--20
макс.абс.знач.--0.3095605792623588 итер--21
макс.абс.знач.--0.29544878005153236 итер--22
макс.

макс.абс.знач.--0.046129960474385126 итер--90
макс.абс.знач.--0.04544466739806891 итер--91
макс.абс.знач.--0.04477187350888418 итер--92
макс.абс.знач.--0.04411126557063516 итер--93
макс.абс.знач.--0.04346254120105782 итер--94
макс.абс.знач.--0.0428254083934072 итер--95
макс.абс.знач.--0.042199585062813516 итер--96
макс.абс.знач.--0.04158479861600668 итер--97
макс.абс.знач.--0.04098078554305747 итер--98
макс.абс.знач.--0.04038729102990005 итер--99
макс.абс.знач.--0.03980406859043565 итер--100
макс.абс.знач.--0.039230879717121046 итер--101
макс.абс.знач.--0.03866749354899249 итер--102
макс.абс.знач.--0.03811368655614978 итер--103
макс.абс.знач.--0.03756924223977134 итер--104
макс.абс.знач.--0.037033950846811846 итер--105
макс.абс.знач.--0.036507609098540734 итер--106
макс.абс.знач.--0.03599001993218177 итер--107
макс.абс.знач.--0.03548099225492964 итер--108
макс.абс.знач.--0.034980340709648464 итер--109
макс.абс.знач.--0.03448788545164348 итер--110
макс.абс.знач.--0.03400345193589294 ите

макс.абс.знач.--0.028834847736959796 итер--125
макс.абс.знач.--0.02850156135233126 итер--126
макс.абс.знач.--0.028171418196264775 итер--127
макс.абс.знач.--0.02784441471737996 итер--128
макс.абс.знач.--0.027520546164017303 итер--129
макс.абс.знач.--0.02719980664473567 итер--130
макс.абс.знач.--0.0268821891861994 итер--131
макс.абс.знач.--0.02656768578858821 итер--132
макс.абс.знач.--0.02625628747861618 итер--133
макс.абс.знач.--0.02594798436030302 итер--134
макс.абс.знач.--0.025642765663576705 итер--135
макс.абс.знач.--0.025340619790809017 итер--136
макс.абс.знач.--0.02504153436138756 итер--137
макс.абс.знач.--0.0247454962543976 итер--138
макс.абс.знач.--0.024452491649504658 итер--139
макс.абс.знач.--0.02416250606611936 итер--140
макс.абс.знач.--0.02387552440090743 итер--141
макс.абс.знач.--0.023591530963734165 итер--142
макс.абс.знач.--0.023310509512086606 итер--143
макс.абс.знач.--0.02303244328407005 итер--144
макс.абс.знач.--0.0227573150300074 итер--145
макс.абс.знач.--0.02248510704

макс.абс.знач.--0.02041031880274773 итер--154
макс.абс.знач.--0.02016353434748826 итер--155
макс.абс.знач.--0.019919473742830866 итер--156
макс.абс.знач.--0.01967811591942769 итер--157
макс.абс.знач.--0.01943943962830494 итер--158
макс.абс.знач.--0.01920342345814573 итер--159
макс.абс.знач.--0.018970045851829955 итер--160
макс.абс.знач.--0.01873928512226294 итер--161
макс.абс.знач.--0.018511119467516265 итер--162
макс.абс.знач.--0.018285526985312755 итер--163
макс.абс.знач.--0.01806248568687683 итер--164
макс.абс.знач.--0.01784197351018168 итер--165
макс.абс.знач.--0.01762396833260274 итер--166
макс.абс.знач.--0.017408447983022653 итер--167
макс.абс.знач.--0.01719539025338662 итер--168
макс.абс.знач.--0.01698477290973553 итер--169
макс.абс.знач.--0.01677657370275457 итер--170
макс.абс.знач.--0.016570770377830505 итер--171
макс.абс.знач.--0.0163673406846438 итер--172
макс.абс.знач.--0.01616626238633492 итер--173
макс.абс.знач.--0.015967513268221083 итер--174
макс.абс.знач.--0.0157710711

макс.абс.знач.--0.015007930434954573 итер--179
макс.абс.знач.--0.014822692131503704 итер--180
макс.абс.знач.--0.014639628777431026 итер--181
макс.абс.знач.--0.014458718603650575 итер--182
макс.абс.знач.--0.014279939925118737 итер--183
макс.абс.знач.--0.014103271146100129 итер--184
макс.абс.знач.--0.013928690765155098 итер--185
макс.абс.знач.--0.013756177379864804 итер--186
макс.абс.знач.--0.013585709691291806 итер--187
макс.абс.знач.--0.013417266508209328 итер--188
макс.абс.знач.--0.013250826751074654 итер--189
макс.абс.знач.--0.013086369455788185 итер--190
макс.абс.знач.--0.012923873777233736 итер--191
макс.абс.знач.--0.012763318992598091 итер--192
макс.абс.знач.--0.012604684504503146 итер--193
макс.абс.знач.--0.012447949843931915 итер--194
макс.абс.знач.--0.012293094672971454 итер--195
макс.абс.знач.--0.012140098787384819 итер--196
макс.абс.знач.--0.01198894211899718 итер--197
макс.абс.знач.--0.011839604737926496 итер--198
макс.абс.знач.--0.011692066854657856 итер--199
макс.абс.знач.

...

макс.абс.знач.--0.057242624233938465 итер--76
макс.абс.знач.--0.056341977395892855 итер--77
макс.абс.знач.--0.055459696487615597 итер--78
макс.абс.знач.--0.0545952479550618 итер--79
макс.абс.знач.--0.053748119441330594 итер--80
макс.абс.знач.--0.052917818751695496 итер--81
макс.абс.знач.--0.052103872875313 итер--82
макс.абс.знач.--0.05130582706041264 итер--83
макс.абс.знач.--0.050523243939924786 итер--84
макс.абс.знач.--0.04975570270467076 итер--85
макс.абс.знач.--0.04900279832140294 итер--86
макс.абс.знач.--0.048264140793091624 итер--87
макс.абс.знач.--0.04753935445906502 итер--88
макс.абс.знач.--0.04682807733268443 итер--89
макс.абс.знач.--0.046129960474385126 итер--90
макс.абс.знач.--0.04544466739806891 итер--91
макс.абс.знач.--0.04477187350888418 итер--92
макс.абс.знач.--0.04411126557063516 итер--93
макс.абс.знач.--0.04346254120105782 итер--94
макс.абс.знач.--0.0428254083934072 итер--95
макс.абс.знач.--0.042199585062813516 итер--96
макс.абс.знач.--0.04158479861600668 итер--97
макс.

макс.абс.знач.--0.06739415321345693 итер--66
макс.абс.знач.--0.06627496670966239 итер--67
макс.абс.знач.--0.06518092303460804 итер--68
макс.абс.знач.--0.0641112046253992 итер--69
макс.абс.знач.--0.06306502941501424 итер--70
макс.абс.знач.--0.06204164901512292 итер--71
макс.абс.знач.--0.061040346997485825 итер--72
макс.абс.знач.--0.06006043726892293 итер--73
макс.абс.знач.--0.05910126253502767 итер--74
макс.абс.знач.--0.05816219284797195 итер--75
макс.абс.знач.--0.057242624233938465 итер--76
макс.абс.знач.--0.056341977395892855 итер--77
макс.абс.знач.--0.055459696487615597 итер--78
макс.абс.знач.--0.0545952479550618 итер--79
макс.абс.знач.--0.053748119441330594 итер--80
макс.абс.знач.--0.052917818751695496 итер--81
макс.абс.знач.--0.052103872875313 итер--82
макс.абс.знач.--0.05130582706041264 итер--83
макс.абс.знач.--0.050523243939924786 итер--84
макс.абс.знач.--0.04975570270467076 итер--85
макс.абс.знач.--0.04900279832140294 итер--86
макс.абс.знач.--0.048264140793091624 итер--87
макс.а

макс.абс.знач.--0.341222728455631 итер--19
макс.абс.знач.--0.32476160744359206 итер--20
макс.абс.знач.--0.3095605792623588 итер--21
макс.абс.знач.--0.29544878005153236 итер--22
макс.абс.знач.--0.28228795913399113 итер--23
макс.абс.знач.--0.26996522810278245 итер--24
макс.абс.знач.--0.25838755990546314 итер--25
макс.абс.знач.--0.2474775920247307 итер--26
макс.абс.знач.--0.23717040585400814 итер--27
макс.абс.знач.--0.22741104030626413 итер--28
макс.абс.знач.--0.2181525604538355 итер--29
макс.абс.знач.--0.20935454794791997 итер--30
макс.абс.знач.--0.20098191371075577 итер--31
макс.абс.знач.--0.19300395825396918 итер--32
макс.абс.знач.--0.18539362335653695 итер--33
макс.абс.знач.--0.17812689247643482 итер--34
макс.абс.знач.--0.17118230743486035 итер--35
макс.абс.знач.--0.1645405765196609 итер--36
макс.абс.знач.--0.1581842548750055 итер--37
макс.абс.знач.--0.15209748236641613 итер--38
макс.абс.знач.--0.14626576739239205 итер--39
макс.абс.знач.--0.1406758076192722 итер--40
макс.абс.знач.--0.

макс.абс.знач.--0.01806248568687683 итер--164
макс.абс.знач.--0.01784197351018168 итер--165
макс.абс.знач.--0.01762396833260274 итер--166
макс.абс.знач.--0.017408447983022653 итер--167
макс.абс.знач.--0.01719539025338662 итер--168
макс.абс.знач.--0.01698477290973553 итер--169
макс.абс.знач.--0.01677657370275457 итер--170
макс.абс.знач.--0.016570770377830505 итер--171
макс.абс.знач.--0.0163673406846438 итер--172
макс.абс.знач.--0.01616626238633492 итер--173
макс.абс.знач.--0.015967513268221083 итер--174
макс.абс.знач.--0.015771071146122428 итер--175
макс.абс.знач.--0.015576913874281279 итер--176
макс.абс.знач.--0.015385019352906074 итер--177
макс.абс.знач.--0.015195365535354237 итер--178
макс.абс.знач.--0.015007930434954573 итер--179
макс.абс.знач.--0.014822692131503704 итер--180
макс.абс.знач.--0.014639628777431026 итер--181
макс.абс.знач.--0.014458718603650575 итер--182
макс.абс.знач.--0.014279939925118737 итер--183
макс.абс.знач.--0.014103271146100129 итер--184
макс.абс.знач.--0.0139

макс.абс.знач.--0.019919473742830866 итер--156
макс.абс.знач.--0.01967811591942769 итер--157
макс.абс.знач.--0.01943943962830494 итер--158
макс.абс.знач.--0.01920342345814573 итер--159
макс.абс.знач.--0.018970045851829955 итер--160
макс.абс.знач.--0.01873928512226294 итер--161
макс.абс.знач.--0.018511119467516265 итер--162
макс.абс.знач.--0.018285526985312755 итер--163
макс.абс.знач.--0.01806248568687683 итер--164
макс.абс.знач.--0.01784197351018168 итер--165
макс.абс.знач.--0.01762396833260274 итер--166
макс.абс.знач.--0.017408447983022653 итер--167
макс.абс.знач.--0.01719539025338662 итер--168
макс.абс.знач.--0.01698477290973553 итер--169
макс.абс.знач.--0.01677657370275457 итер--170
макс.абс.знач.--0.016570770377830505 итер--171
макс.абс.знач.--0.0163673406846438 итер--172
макс.абс.знач.--0.01616626238633492 итер--173
макс.абс.знач.--0.015967513268221083 итер--174
макс.абс.знач.--0.015771071146122428 итер--175
макс.абс.знач.--0.015576913874281279 итер--176
макс.абс.знач.--0.01538501